## ActiveSVM

### data要件

- indir
"./cepstrums/
- outmodel
"./SVMfile/day/model/"
- car_ids
["001","005","010","011","013"]
- 4_of_cepstrums
"./SVMfile/20180814/4_of_cepstrums/"

In [6]:
indir = "./cepstrums/"
car_ids =  ["001","005","010","011","013"]
without_no_label = False
c4 = "./SVMfile/20180814/4_of_cepstrums/"

In [2]:
import itertools
directory_list = ["001", "005", "010", "011", "013"]
combi_list = list(itertools.combinations(directory_list,4))

#### create_model

In [3]:
import sklearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

from sklearn.datasets import load_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

In [4]:
kernel = "rbf"

In [5]:
import Acreate_svm_model
import pandas as pd
from os import path
for combi in combi_list:
    target = [i for i in directory_list if not i in combi]
    xdata, xlabel = Acreate_svm_model.read_datas(indir, combi, without_no_label = without_no_label)
    print("DONE load train data:", xdata.shape, xlabel.shape)
    model = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=1e-3)
    model.fit(xdata, xlabel)
    print("DONE fit")
    outmodel = "./SVMfile/20180814/model/%s.svm"%target[0]
    modeldir = path.dirname(outmodel)
    if not modeldir is None and len(modeldir) > 0 and not path.isdir(modeldir): os.makedirs(modeldir)
    joblib.dump(model, outmodel)
    print(model.score(xdata, xlabel))

['./cepstrums/001/*.label.csv', './cepstrums/005/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/011/*.label.csv']
DONE load train data: (181602, 180) (181602,)
DONE fit
0.7649860684353696
['./cepstrums/001/*.label.csv', './cepstrums/005/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/013/*.label.csv']
DONE load train data: (193264, 180) (193264,)
DONE fit
0.763535888732511
['./cepstrums/001/*.label.csv', './cepstrums/005/*.label.csv', './cepstrums/011/*.label.csv', './cepstrums/013/*.label.csv']
DONE load train data: (178144, 180) (178144,)
DONE fit
0.7402831417280402
['./cepstrums/001/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/011/*.label.csv', './cepstrums/013/*.label.csv']
DONE load train data: (198535, 180) (198535,)
DONE fit
0.7668773767849497
['./cepstrums/005/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/011/*.label.csv', './cepstrums/013/*.label.csv']
DONE load train data: (227383, 180) (227383,)
DONE fit
0.7566836570895802


#### predict

In [53]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sklearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

from sklearn.datasets import load_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

In [54]:
import Acreate_svm_model
xdata, xlabel = Acreate_svm_model.read_datas("./cepstrums/", ["001"], without_no_label = without_no_label)
# 予測モデルを復元
clf = joblib.load("./SVMfile/20180814/model/001.svm") 

['./cepstrums/001/*.label.csv']


In [10]:
eval_car_ids = ["001"]
outeval = "./test.csv"

In [55]:
print(clf.score(xdata,xlabel))

0.75912156320249


In [15]:
import glob
import pandas as pd
from Acreate_svm_model import read_csv
outpredict = "./SVMfile/20180814/predictdata/"
outdecision = "./SVMfile/20180815/decision/"
for i in car_ids:
    for _label in glob.glob(indir + "%s/*.label.csv" % i, recursive = True):
        _accel = _label.replace("label", "accel")
        _gyro = _label.replace("label", "gyro")
        _edata, _elabel = read_csv(_accel, _gyro, _label)#変更
        
        model = joblib.load("./SVMfile/20180814/model/%s.svm" % i)
        
        #predicted
        decision = model.decision_function(_edata)
        predicted = model.predict(_edata)
        #predictedcsv保存
        predicted_csv = pd.DataFrame(predicted)
        decision_csv = pd.DataFrame(decision)
        _label = _label.replace("/home/aj1m0n/jupyter_notebook/Datamining/20180731/cepstrums/", "-").replace(".label", "").replace("/","")
        
        predicted_csv.to_csv("%spredicted%s" %(outpredict, _label), index=False)#csv保存
        decision_csv.to_csv("%spredicted%s" %(outdecision, _label), index=False)

##### read predict of lstm

In [2]:
import pandas as pd
svm0406 = pd.read_csv("./SVMfile/20180814/predictdata/predicted.cepstrums001001_20170406_000.csv.csv")
lstm0406 = pd.read_csv("./models/20180806/predict/predicted-001001_20170406_000.csv")

In [3]:
import numpy as np
lstm = lstm0406[["NO_LABEL", "ROLL", "RUN", "DOOR"]].values
lstm = np.argmax(lstm, axis = -1)

In [4]:
lstm

array([0, 0, 0, ..., 0, 0, 0])

In [5]:
svm = np.array(svm0406["0"])

In [6]:
len(lstm)

7796

In [7]:
len(svm)

7798

In [36]:
not_match = []
for i in range(len(lstm)):
    if svm[i] != lstm[i]:
        not_match.append(i)


#### and rangue

In [9]:
rowclass = pd.read_csv("./models/20180806/class/0.1/0.1-001001_20170406_000.csv")
timelist = []
for l in not_match:
    timelist.append(lstm0406.iloc[l,4])

In [10]:
testtest = [i for i in rowclass["timestamp"] if i in timelist]

#### and no rague

In [68]:
import pandas as pd
#range100 = pd.DataFrame(index=["timestamp"], columns=["car_id","NO_LABEL", "ROLL", "RUN", "DOOR"])
testn = 0
modelist = []
prelstm = lstm0406.set_index("timestamp")
for index, row in prelstm.iterrows():
    for time in timelist:
        if index == time:
            if testn - 10 < 0:
                rowl = prelstm[:20][["NO_LABEL", "ROLL", "RUN", "DOOR"]].values
                rows = svm[:20]
            elif testn + 25 > len(rowdata):
                rowl = prelstm[len(prelstm) - 20:][["NO_LABEL", "ROLL", "RUN", "DOOR"]].values
                rows = svm[len(prelstm) - 20 : len(prelstm)]
            else:
                #range100 = pd.concat([range100, rowdata[testn - 10: testn + 10]])
                rowl= prelstm[testn - 10: testn + 10][["NO_LABEL", "ROLL", "RUN", "DOOR"]].values
                rows = svm[testn - 10: testn + 10]
            rowl = np.argmax(rowl, axis = -1)
            judges = [0,0,0,0]
            judgel = [0,0,0,0]
            for j in rows:
                judges[j] += 1
            for j in rowl:
                judgel[j] += 1
            count = np.array(judges) + np.array(judgel)
            mode = np.argmax(count)
            modelist.append(mode)
    testn += 1

In [70]:
len(timelist)

2357

In [71]:
len(modelist)

2357

In [72]:
test2 = np.argmax(prelstm[["NO_LABEL", "ROLL", "RUN", "DOOR"]].values, axis = -1)

for index, row in prelstm.iterrows():
    t = 0
    for time in timelist:
        if index == time:
            test2[t] = modelist[t]
        t += 1

In [74]:
len(test2)

7796

#### accuracy

In [73]:
label = pd.read_csv("./cepstrums/001/001_20170406_000.label.csv")
label = np.argmax(label[["NO_LABEL", "ROLL", "RUN", "DOOR"]].values, axis = -1)

In [78]:
accuracylist = []
accuracylist2 = []
for itest in range(len(test2)):
    if test2[itest] != label[itest]:
        accuracylist.append(itest)
    if svm[itest] != label[itest]:
        accuracylist2.append(itest)

In [81]:
len(accuracylist2)/len(test2)

0.26975371985633656

In [82]:
len(accuracylist)/len(test2)

0.39648537711647

### 境界線との距離

In [32]:
import sklearn
import pandas as pd
import numpy as np

predata = pd.read_csv("./SVMfile/20180814/predictdata/predicted.cepstrums001001_20170406_000.csv")
decdata = pd.read_csv("./SVMfile/20180815/decision/predicted.cepstrums001001_20170406_000.csv")
predata = np.array(predata["0"])
tt = 0
decisionlist = []
for index, row in decdata.iterrows():
    k = predata[tt]
    decisionlist.append(row[k])
    tt += 1

In [33]:
decisionlist

[0.43116412918870506,
 0.8718217544944783,
 0.8419636168160701,
 1.2522872842994532,
 1.39772430206306,
 1.3496110426386971,
 0.533279710267297,
 0.8055804333155571,
 1.4478729674270614,
 1.199851067833312,
 1.3264352776246011,
 1.0491366656574697,
 0.5273569528525983,
 0.8118815517314991,
 0.9627060209728548,
 1.3544359547172666,
 0.6300378385540935,
 0.6812618997017316,
 1.0679747276684268,
 0.5824613310443888,
 1.1304419974459117,
 1.1503152839404591,
 1.8193055187967728,
 0.3124781157189791,
 0.5560996628642743,
 0.3693647500495056,
 0.4520545351391343,
 0.32590056842622817,
 0.5376465476808819,
 0.06419745518531639,
 0.07840614347739727,
 0.6865495797340904,
 0.8817038936441035,
 1.4310811724964507,
 0.4470830158399016,
 1.1357555710173808,
 0.5902022106828883,
 0.9215619495089984,
 0.8779296940985852,
 1.3302889509536624,
 1.5573172301763272,
 1.3706222212029593,
 1.6255868480043276,
 1.1467708954734586,
 1.4144235087966954,
 1.3335389983154171,
 0.6807432095979724,
 0.9953742051

In [34]:
absdecision = []
for i in decisionlist:
    absdecision.append(abs(i))

In [35]:
absdecision

[0.43116412918870506,
 0.8718217544944783,
 0.8419636168160701,
 1.2522872842994532,
 1.39772430206306,
 1.3496110426386971,
 0.533279710267297,
 0.8055804333155571,
 1.4478729674270614,
 1.199851067833312,
 1.3264352776246011,
 1.0491366656574697,
 0.5273569528525983,
 0.8118815517314991,
 0.9627060209728548,
 1.3544359547172666,
 0.6300378385540935,
 0.6812618997017316,
 1.0679747276684268,
 0.5824613310443888,
 1.1304419974459117,
 1.1503152839404591,
 1.8193055187967728,
 0.3124781157189791,
 0.5560996628642743,
 0.3693647500495056,
 0.4520545351391343,
 0.32590056842622817,
 0.5376465476808819,
 0.06419745518531639,
 0.07840614347739727,
 0.6865495797340904,
 0.8817038936441035,
 1.4310811724964507,
 0.4470830158399016,
 1.1357555710173808,
 0.5902022106828883,
 0.9215619495089984,
 0.8779296940985852,
 1.3302889509536624,
 1.5573172301763272,
 1.3706222212029593,
 1.6255868480043276,
 1.1467708954734586,
 1.4144235087966954,
 1.3335389983154171,
 0.6807432095979724,
 0.9953742051

In [36]:
max(absdecision)

3.0753051891514684

In [37]:
min(absdecision)

0.0002118542980404925

In [44]:
import pandas as pd
abd = pd.DataFrame(columns=["decision","timestamp"])
cep = pd.read_csv("./cepstrums/001/001_20170406_000.label.csv")
abd["timestamp"] = cep["timestamp"]
abd["decision"] = absdecision
abd = abd.set_index("timestamp")

In [45]:
abd.head()

,decision
timestamp,
2017-04-06T07:26:33.338,0.431164
2017-04-06T07:26:34.338,0.871822
2017-04-06T07:26:35.338,0.841964
2017-04-06T07:26:36.338,1.252287
2017-04-06T07:26:37.338,1.397724


In [3]:
import Acreate_svm_model
import pandas as pd
from os import path
from sklearn import svm
directory_list =  ["001","005","010","011","013"]
def testSVM(combi_list: list, indir: str, outmodel: str):
    for combi in combi_list:
        #target = [i for i in directory_list if not i in combi]
        xdata, xlabel = Acreate_svm_model.read_datas(indir, combi, without_no_label = False)
        print("DONE load train data:", xdata.shape, xlabel.shape)
        model = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=1e-3)
        #model = svm.SVC(kernel = "sigmoid")
        model.fit(xdata, xlabel)
        print("DONE fit")
        outmodel = "./SVMfile/20180814/model/test.svm"
        modeldir = path.dirname(outmodel)
        if not modeldir is None and len(modeldir) > 0 and not path.isdir(modeldir): os.makedirs(modeldir)
        joblib.dump(model, outmodel)
        print(model.score(xdata, xlabel))

In [ ]:
testSVM([["001","005","010","011","013"]], "./cepstrums/yugo/", "./SVMfile/20180815/ymodel/")

['./cepstrums/yugo/001/*.label.csv', './cepstrums/yugo/005/*.label.csv', './cepstrums/yugo/010/*.label.csv', './cepstrums/yugo/011/*.label.csv', './cepstrums/yugo/013/*.label.csv']
DONE load train data: (235181, 180) (235181,)


In [51]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sklearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

from sklearn.datasets import load_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

In [ ]:
import Acreate_svm_model
xdata, xlabel = Acreate_svm_model.read_datas("./cepstrums/", ["001"], without_no_label = without_no_label)
# 予測モデルを復元
clf = joblib.load("./SVMfile/20180814/model/test.svm")
print(clf.score(xdata,xlabel))

# 検証実験

In [1]:
indir = "./cepstrums/"
car_ids =  ["001","005","010","011","013"]
without_no_label = False
c4 = "./SVMfile/20180814/4_of_cepstrums/"

In [2]:
import itertools
directory_list = ["001", "005", "010", "011", "013"]
combi_list = list(itertools.combinations(directory_list,4))

In [30]:
from os import path
import os
import glob
import Acreate_svm_model
import sklearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

from sklearn.datasets import load_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

file_ptns = []
for combi in combi_list:
    target = [i for i in directory_list if not i in combi]
    file_ptns = [path.join(indir, "%s/*.label.csv" % i) for i in combi]
    test = glob.glob("./cepstrums/%s/*.label.csv" %target[0])
    for i in test:
        file_ptns.append(i)
        xdata, xlabel = Acreate_svm_model.read_datas(indir, file_ptns, without_no_label = without_no_label)
        print("DONE load train data:", xdata.shape, xlabel.shape)
        model = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=1e-3)
        model.fit(xdata, xlabel)
        print("DONE fit")
        outmodel = "./SVMfile/20180815/ymodel/%s/%s.svm"%(target[0],i.replace(".csv",""))
        modeldir = path.dirname(outmodel)
        if not modeldir is None and len(modeldir) > 0 and not path.isdir(modeldir): os.makedirs(modeldir)
        joblib.dump(model, outmodel)
        print(model.score(xdata, xlabel))
        file_ptns.remove(i)

['./cepstrums/001/*.label.csv', './cepstrums/005/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/011/*.label.csv', './cepstrums/013/013_20170407_000.label.csv']
DONE load train data: (194969, 180) (194969,)
DONE fit
0.7637419282039709
['./cepstrums/001/*.label.csv', './cepstrums/005/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/011/*.label.csv', './cepstrums/013/013_20170406_000.label.csv']
DONE load train data: (199519, 180) (199519,)
DONE fit
0.7601030478300312
['./cepstrums/001/*.label.csv', './cepstrums/005/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/011/*.label.csv', './cepstrums/013/013_20170511_000.label.csv']
DONE load train data: (197592, 180) (197592,)
DONE fit
0.7659216972347058
['./cepstrums/001/*.label.csv', './cepstrums/005/*.label.csv', './cepstrums/010/*.label.csv', './cepstrums/011/*.label.csv', './cepstrums/013/013_20170512_000.label.csv']
DONE load train data: (197458, 180) (197458,)
DONE fit
0.7616049995442069
['./cepstrums/001/*.

## 交差検証

### SVMacc

In [9]:
import Acreate_svm_model
import pandas as pd
import glob
from sklearn.externals import joblib
from sklearn import cross_validation
from statistics import mean
car_ids = ["001","005","010","011","013"]

#### 別日のデータを入れた場合

In [10]:
for carid in car_ids:
    filelist = glob.glob("./cepstrums/%s/*.label.csv"%carid)
    scorelist = []
    for fptn in filelist:
        X_data, y_data = Acreate_svm_model.read_datas("./cepstrums/", [fptn], without_no_label = False)
        #labelday = fptn.replace("./cepstrums/%s/"%carid,"").replace(".csv","")
        modeldays = [i for i in filelist if not i in [fptn]]
        testresult = []
        for modelday in modeldays:
            modelday = modelday.replace("./cepstrums/%s/"%carid,"").replace(".csv","")
            model = joblib.load("./SVMfile/20180815/ymodel/%s/cepstrums/%s/%s.svm"%(carid,carid,modelday))
            testresult.append(model.score(X_data,y_data))
        scorelist.append(mean(testresult))
    resultcsv = pd.DataFrame(columns=["day","score"])
    resultcsv["day"] = filelist
    resultcsv["score"] = scorelist
    resultcsv = resultcsv.set_index("day")
    resultcsv.to_csv("./SVMfile/20180816/SVMresult/ALLscrore-%s.csv"%carid)

['./cepstrums/001/001_20170407_000.label.csv']
['./cepstrums/001/001_20170406_000.label.csv']
['./cepstrums/005/005_20170406_000.label.csv']
['./cepstrums/005/005_20170511_000.label.csv']
['./cepstrums/005/005_20170511_001.label.csv']
['./cepstrums/005/005_20170407_000.label.csv']
['./cepstrums/010/010_20170512_000.label.csv']
['./cepstrums/010/010_20170407_000.label.csv']
['./cepstrums/010/010_20170406_000.label.csv']
['./cepstrums/010/010_20170511_000.label.csv']
['./cepstrums/011/011_20170407_000.label.csv']
['./cepstrums/011/011_20170511_000.label.csv']
['./cepstrums/011/011_20170512_000.label.csv']
['./cepstrums/013/013_20170407_000.label.csv']
['./cepstrums/013/013_20170406_000.label.csv']
['./cepstrums/013/013_20170511_000.label.csv']
['./cepstrums/013/013_20170512_000.label.csv']


#### 別日のデータを入れいていない場合

In [11]:
for carid in car_ids:
    filelist = glob.glob("./cepstrums/%s/*.label.csv"%carid)
    #scorelist = []
    testresult = []
    for fptn in filelist:
        X_data, y_data = Acreate_svm_model.read_datas("./cepstrums/", [fptn], without_no_label = False)
        #labelday = fptn.replace("./cepstrums/%s/"%carid,"").replace(".csv","")
        #modeldays = [i for i in filelist if not i in [fptn]]
        #testresult = []
        #for modelday in modeldays:
        #modelday = modelday.replace("./cepstrums/%s/"%carid,"").replace(".csv","")
        model = joblib.load("./SVMfile/20180814/model/%s.svm"%(carid))
        testresult.append(model.score(X_data,y_data))
    #scorelist.append(mean(testresult))
    resultcsv = pd.DataFrame(columns=["day","score"])
    resultcsv["day"] = filelist
    resultcsv["score"] = testresult
    resultcsv = resultcsv.set_index("day")
    resultcsv.to_csv("./SVMfile/20180816/SVMresult/normalscrore-%s.csv"%carid)

['./cepstrums/001/001_20170407_000.label.csv']
['./cepstrums/001/001_20170406_000.label.csv']
['./cepstrums/005/005_20170406_000.label.csv']
['./cepstrums/005/005_20170511_000.label.csv']
['./cepstrums/005/005_20170511_001.label.csv']
['./cepstrums/005/005_20170407_000.label.csv']
['./cepstrums/010/010_20170512_000.label.csv']
['./cepstrums/010/010_20170407_000.label.csv']
['./cepstrums/010/010_20170406_000.label.csv']
['./cepstrums/010/010_20170511_000.label.csv']
['./cepstrums/011/011_20170407_000.label.csv']
['./cepstrums/011/011_20170511_000.label.csv']
['./cepstrums/011/011_20170512_000.label.csv']
['./cepstrums/013/013_20170407_000.label.csv']
['./cepstrums/013/013_20170406_000.label.csv']
['./cepstrums/013/013_20170511_000.label.csv']
['./cepstrums/013/013_20170512_000.label.csv']


#### plot

In [29]:
plotdf = pd.DataFrame(columns=["carid"])
carlist = []
Ascore = []
nscore = []
for carid in car_ids:
    carlist.append(carid)
    plotcsvs = glob.glob("./SVMfile/20180816/SVMresult/*%s.csv"%carid)
    for plotcsv in plotcsvs:
        if "ALL" in plotcsv:
            #print(plotcsv.find("ALL"))
            Ascore.append(mean(pd.read_csv(plotcsv)["score"]))
            print(Ascore)
        else:
            nscore.append(mean(pd.read_csv(plotcsv)["score"]))
        #plotdf = pd.DataFrame(columns=["carid"])
plotdf["carid"] = carlist
plotdf["Ascore"] = Ascore
plotdf["nscore"] = nscore

[0.7602849877961078]
[0.7602849877961078, 0.7566847044561834]
[0.7602849877961078, 0.7566847044561834, 0.8083589551979165]
[0.7602849877961078, 0.7566847044561834, 0.8083589551979165, 0.7232029137990492]
[0.7602849877961078, 0.7566847044561834, 0.8083589551979165, 0.7232029137990492, 0.7354188998564628]


In [30]:
plotdf

,carid,Ascore,nscore
0,001,0.760285,0.756478
1,005,0.756685,0.756155
2,010,0.808359,0.807574
3,011,0.723203,0.714085
4,013,0.735419,0.733588
